In [2]:
# Helper functions

def head_dict(d, n = 20, digits = 4):
    for key in sorted(d, key=d.get, reverse=True)[0:n]:
        print("%s: %s" % (key, round(d[key], digits)))

def normalize_frequencies(d):
    total = 0
    for key in d:
        total += d[key]
    for key in d:
        d[key] /= total_word_count
    return d

from collections import Counter

text = open('shakespeare.txt').read()
words = text.lower().replace(',', ' ').replace('.', ' ').split()

# Unigram
total_word_count = len(words)
word_frequencies = Counter(words)

unigram = normalize_frequencies(word_frequencies)

head_dict(unigram)

the: 0.0305
and: 0.0296
i: 0.0223
to: 0.0209
of: 0.0201
a: 0.0159
you: 0.0143
my: 0.0138
that: 0.0121
in: 0.012
is: 0.0105
not: 0.0093
for: 0.0091
with: 0.0088
it: 0.008
be: 0.0077
me: 0.0076
your: 0.0076
his: 0.0076
this: 0.007


In [3]:
# Bigram

bigram = {}

for i in range(0, len(words) - 1):
    key = (words[i], words[i+1])
    try:
        bigram[key] += 1
    except KeyError:
        bigram[key] = 1
normalize_frequencies(bigram)
head_dict(bigram)

('i', 'am'): 0.002
('of', 'the'): 0.0019
('in', 'the'): 0.0018
('i', 'have'): 0.0018
('i', 'will'): 0.0017
('to', 'the'): 0.0015
('my', 'lord'): 0.0015
('it', 'is'): 0.0011
('to', 'be'): 0.0011
('that', 'i'): 0.001
('and', 'the'): 0.0009
('i', 'do'): 0.0009
('and', 'i'): 0.0008
('of', 'my'): 0.0008
('you', 'are'): 0.0007
('is', 'the'): 0.0007
('i', 'would'): 0.0007
('the', 'king'): 0.0007
('you', 'have'): 0.0007
('he', 'is'): 0.0007


In [4]:
def shannon_ngram(words, ngram, length=10):
    if length == 0:
        return words
    
    n = len(list(ngram.keys())[0])
    next_word = ''
    max_p = 0
    
    for key in ngram:
        if words[-n+1:] == key[:n-1] and ngram[key] > max_p:
            next_word = key[-1]
            max_p = ngram[key]
            
    return shannon_ngram(words + (next_word,), ngram, length - 1)
#     A = words
#     B = shannon_ngram(next_words, ngram, N-1)
#     print(A)
#     print(B)
#     return A + B

print(shannon_ngram(('if',), bigram))
print(shannon_ngram(('god',), bigram))
print(shannon_ngram(('king',), bigram))
print(shannon_ngram(('hand',), bigram))

('if', 'you', 'are', 'not', 'to', 'the', 'king', 'henry', 'the', 'king', 'henry')
('god', 'save', 'you', 'are', 'not', 'to', 'the', 'king', 'henry', 'the', 'king')
('king', 'henry', 'the', 'king', 'henry', 'the', 'king', 'henry', 'the', 'king', 'henry')
('hand', 'and', 'the', 'king', 'henry', 'the', 'king', 'henry', 'the', 'king', 'henry')


In [5]:
# Trigram

trigram = {}

for i in range(0, len(words) - 2):
    key = (words[i], words[i+1], words[i+2])
    try:
        trigram[key] += 1
    except KeyError:
        trigram[key] = 1
normalize_frequencies(trigram)
head_dict(trigram)

('so', 'long', 'as'): 0.0003
('works', 'of', 'william'): 0.0002
('of', 'william', 'shakespeare'): 0.0002
('complete', 'works', 'of'): 0.0002
('the', 'complete', 'works'): 0.0002
('world', 'library', 'inc'): 0.0002
('of', 'the', 'complete'): 0.0002
('by', 'project', 'gutenberg'): 0.0002
('by', 'any', 'service'): 0.0002
('long', 'as', 'such'): 0.0002
('project', 'gutenberg', 'etext'): 0.0002
('for', 'download', 'time'): 0.0002
('is', 'provided', 'by'): 0.0002
('are', 'for', 'your'): 0.0002
('provided', 'by', 'project'): 0.0002
('for', 'your', 'or'): 0.0002
('and', '(2)', 'are'): 0.0002
('version', 'of', 'the'): 0.0002
('illinois', 'benedictine', 'college'): 0.0002
('machine', 'readable', 'copies'): 0.0002


In [7]:
print(shannon_ngram(('if', 'you'), trigram))
print(shannon_ngram(('the', 'king'), trigram))

('if', 'you', 'will', 'not', 'be', 'so', 'bold', 'to', 'say', 'the', 'truth', 'of')
('the', 'king', 'and', 'queen', 'and', 'her', 'two', 'sons', 'demetrius', 'and', 'chiron', 'with')


In [1]:
# Perplexity

test_data = words[0:round(len(words) * 0.8)]
validation_data = words[round(len(words) * 0.8)+1:]

NameError: name 'words' is not defined